In [1]:
corpus = [
     'this is the first document',
     'this document is the second document',
     'and this is the third one',
     'is this the first document',
]


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(corpus)
skl_output = vectorizer.transform(corpus)

# Task 1  

In [3]:
# Write your code here.
# Make sure its well documented and readble with appropriate comments.
# Compare your results with the above sklearn tfidf vectorizer
# You are not supposed to use any other library apart from the ones given below

from collections import Counter
from tqdm import tqdm
from scipy.sparse import csr_matrix
import math
import operator
from sklearn.preprocessing import normalize
import numpy

In [4]:
def fit(dataset):    
    unique_words = set() 
    if isinstance(dataset, (list,)):
        for row in dataset: 
            for word in row.split(" "): 
                if len(word) < 2:
                    continue
                unique_words.add(word)
        unique_words = sorted(list(unique_words))
        
        vocab = {j:i for i,j in enumerate(unique_words)}
        
        return vocab
    else:
        print("you need to pass list of sentance")

In [5]:
#list(fit(corpus).keys())

## Vocab

In [6]:
unique_words=fit(corpus)
#unique_words

## Custom IDF calculator method

In [7]:
def calculate_idf(unique_words):
    idf=[]
    for word in unique_words:
        cnt=0
        for row in corpus:
            if word in row:
                cnt+=1
        idf_value=1+math.log((1+len(corpus))/(1+cnt))
        idf.append(idf_value)
    return idf


## Comparing IDF value of sklearn and custom method

In [8]:
idf=calculate_idf(list(fit(corpus).keys()))
print(idf)
print(vectorizer.idf_)

[1.916290731874155, 1.2231435513142097, 1.5108256237659907, 1.0, 1.916290731874155, 1.916290731874155, 1.0, 1.916290731874155, 1.0]
[1.91629073 1.22314355 1.51082562 1.         1.91629073 1.91629073
 1.         1.91629073 1.        ]


## Custom Transform method

In [9]:
def transform(dataset,unique_words,idf):
    rows=[]
    columns=[]
    values=[]
    if isinstance(dataset, (list,)):
        for idx, row in enumerate(tqdm(dataset)): # for each document in the dataset
            # it will return a dict type object where key is the word and values is its frequency, {word:frequency}
            word_freq = dict(Counter(row.split()))
            # for every unique word in the document
            for word, freq in word_freq.items():  # for each unique word in the review.                
                if len(word) < 2:
                    continue
                # we will check if its there in the vocabulary that we build in fit() function
                # dict.get() function will return the values, if the key doesn't exits it will return -1
                col_index = unique_words.get(word, -1) # retreving the dimension number of a word
                
                # if the word exists
                if col_index !=-1:
                    # we are storing the index of the document
                    rows.append(idx)
                    # we are storing the dimensions of the word
                    columns.append(col_index)
                    # we are storing the frequency of the word
                    tf=freq/len(row)
                    #print("tf=",tf)
                    #print("col index=",col_index)
                    idf_val=idf[col_index]
                    #print("idf=",idf_val)
                    tf_idf=tf*idf_val
                    values.append(tf_idf)
        csr=csr_matrix((values, (rows,columns)), shape=(len(dataset),len(unique_words)))
        normalize_vector=normalize(csr,norm='l2')
        
        #print(skl_output[0])
        #print("csr=",csr)
        #print("norm=",normalize_vector)
        return normalize_vector
    else:
        print("you need to pass list of strings")
    

## Comparing the result of sklearn's tranform method and custorm tranform function

In [10]:

print(skl_output[0].toarray())
print(transform(corpus,unique_words,idf)[0].toarray())

[[0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]]


100%|████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<?, ?it/s]

[[0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]]


# Task 2

In [11]:
import pickle
with open('cleaned_strings', 'rb') as f:
    corpus = pickle.load(f)
    
# printing the length of the corpus loaded
print("Number of documents in corpus = ",len(corpus))

Number of documents in corpus =  746


## Custom fit method for Task2 containing all unique words

In [12]:
def fit(dataset):    
    unique_words = set() 
    if isinstance(dataset, (list,)):
        for row in dataset: 
            for word in row.split(" "): 
                if len(word) < 2:
                    continue
                unique_words.add(word)
        unique_words = sorted(list(unique_words))
        
        vocab = {j:i for i,j in enumerate(unique_words)}
        
        return vocab
    else:
        print("you need to pass list of sentance")

In [13]:
unique_words=fit(corpus)
#print(unique_words)

## Custom IDF calculator method for task-2 containin top 50 features (idf values) and vocab

In [27]:
def calculate_idf(unique_words):
    idf=[]
    uni_word=[]
    for word in unique_words:
        cnt=0
        for row in corpus:
            if word in row:
                cnt+=1
        idf_value=1+math.log((1+len(corpus))/(1+cnt))
        uni_word.append(word)
        idf.append(idf_value)
        #print("idf=",len(idf))
        #print("unique word=",len(uni_word))
    #print(len(idf))
    #print(uni_word)
    #print("len uni=",len(uni_word))
    #print("len idf=",len(idf))
    cnt=0
    #print(max(idf))
    """for i in idf:
        if i==max(idf):
            cnt+=1
    #print(cnt)"""
    for i in range(len(idf)):
        for j in range(len(idf)):
            if(idf[i]>idf[j]):
                idf[j],idf[i]=idf[i],idf[j]
            
                uni_word[j],uni_word[i]=uni_word[i],uni_word[j]
        #print(idf[:50])
        #print(uni_word[:50])
    top_50_idf=idf[:50]
        #print("idf=",len(top_50_idf))
    vocab={j:i for i,j in enumerate(uni_word[:50])}
    #print("vocab=",len(vocab))
        #for i in range(len(vocab)):
            #print("word=",list(vocab.keys())[i],",idf=",top_50_idf[i])
    return (top_50_idf,vocab)


In [28]:
#print(len(unique_words))
#print(len(calculate_idf(list(fit(corpus).keys()))))
top_50_idf,vocab=calculate_idf(list(fit(corpus).keys()))
#print(top_50_idf)
#print(vocab)

## Custom Transform method for Task-2 where tf*idf value is calculated for whole corpus based on top features

In [29]:
def transform(dataset,unique_words,idf):
    rows=[]
    columns=[]
    values=[]
    if isinstance(dataset, (list,)):
        for idx, row in enumerate(tqdm(dataset)): # for each document in the dataset
            # it will return a dict type object where key is the word and values is its frequency, {word:frequency}
            word_freq = dict(Counter(row.split()))
            # for every unique word in the document
            for word, freq in word_freq.items():  # for each unique word in the review.                
                if len(word) < 2:
                    continue
                # we will check if its there in the vocabulary that we build in fit() function
                # dict.get() function will return the values, if the key doesn't exits it will return -1
                col_index = unique_words.get(word, -1) # retreving the dimension number of a word
                
                # if the word exists
                if col_index !=-1:
                    # we are storing the index of the document
                    rows.append(idx)
                    # we are storing the dimensions of the word
                    columns.append(col_index)
                    # we are storing the frequency of the word
                    tf=freq/len(row)
                    #print("tf=",tf)
                    #print("col index=",col_index)
                    idf_val=idf[col_index]
                    #print("idf=",idf_val)
                    tf_idf=tf*idf_val
                    values.append(tf_idf)
        csr=csr_matrix((values, (rows,columns)), shape=(len(dataset),len(unique_words)))
        normalize_vector=normalize(csr,norm='l2')
        return normalize_vector
    else:
        print("you need to pass list of strings")
    

## Shape of custom normalized vector

In [30]:
print(transform(corpus,vocab,top_50_idf).shape)

100%|█████████████████████████████████████████████████████████████████████████████| 746/746 [00:00<00:00, 65731.50it/s]

(746, 50)


## Observation->
As there are 746 documents in the corpus and the number of features are 50 Shape=(746,50)

## Output of a single document in your collection of documents

In [26]:
print(transform(corpus,vocab,top_50_idf)[0].toarray())
#print(skl_output[0].toarray())

100%|█████████████████████████████████████████████████████████████████████████████| 746/746 [00:00<00:00, 83385.32it/s]

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]]
